# Favorita Hierarchical Baselines

This notebooks runs and saves the forecasts of hierarchical statistical baseline methods.

- It reads a preprocessed Favorita dataset as defined in [datasetsforecast.favorita](https://github.com/Nixtla/datasetsforecast/blob/feat/favorita_dataset/nbs/favorita.ipynb).
- It filters the dataset by `item_id`.
- It fits base forecasts using StatsForecast's `AutoARIMA`.
- It reconciles the geographic aggregation levels using HierarchicalForecast's baselines, (10 bootstrap seeds).
- It saves the reconciled forecasts, test, and train subsets for each `item_id` into its respective folder `./data/Favorita200/item_id/`

In [1]:
import os
import argparse

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# %%capture
# !pip install statsforecast
# !pip install hierarchicalforecast
# !pip install git+https://github.com/Nixtla/datasetsforecast.git@feat/favorita_dataset

In [3]:
from statsforecast.core import StatsForecast
from statsforecast.models import AutoARIMA, Naive

from hierarchicalforecast.core import HierarchicalReconciliation
from hierarchicalforecast.evaluation import HierarchicalEvaluation
from hierarchicalforecast.methods import BottomUp, TopDown, MinTrace, ERM

from hierarchicalforecast.utils import is_strictly_hierarchical
from hierarchicalforecast.utils import HierarchicalPlot, CodeTimer
from hierarchicalforecast.evaluation import scaled_crps, msse, energy_score

from datasetsforecast.favorita import FavoritaData, FavoritaInfo

import warnings
# Avoid pandas fragmentation warning and positive definite warning
warnings.filterwarnings("ignore")

/Users/kin/anaconda3/envs/hierarchical_baselines/lib/python3.10/site-packages/statsforecast/core.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [4]:
class FavoritaHierarchicalDataset(object):
    # Class with loading, processing and
    # prediction evaluation methods for hierarchical data

    available_datasets = ['Favorita200', 'Favorita500', 'FavoritaComplete']

    @staticmethod
    def _get_hierarchical_scrps(hier_idxs, Y, Yq_hat, q_to_pred):
        # We use the indexes obtained from the aggregation tags
        # to compute scaled CRPS across the hierarchy levels
        # # [n_items, n_stores, n_time, n_quants] 
        scrps_list = []
        for idxs in hier_idxs:
            y      = Y[:, idxs, :]
            yq_hat = Yq_hat[:, idxs, :, :]
            scrps  = scaled_crps(y, yq_hat, q_to_pred)
            scrps_list.append(scrps)
        return scrps_list

    @staticmethod
    def _get_hierarchical_msse(hier_idxs, Y, Y_hat, Y_train):
        # We use the indexes obtained from the aggregation tags
        # to compute scaled CRPS across the hierarchy levels         
        msse_list = []
        for idxs in hier_idxs:
            y       = Y[:, idxs, :]
            y_hat   = Y_hat[:, idxs, :]
            y_train = Y_train[:, idxs, :]
            crps    = msse(y, y_hat, y_train)
            msse_list.append(crps)
        return msse_list    

    @staticmethod
    def _sort_hier_df(Y_df, S_df):
        # NeuralForecast core, sorts unique_id lexicographically
        # deviating from S_df, this class matches S_df and Y_hat_df order.
        Y_df.unique_id = Y_df.unique_id.astype('category')
        Y_df.unique_id = Y_df.unique_id.cat.set_categories(S_df.index)
        Y_df = Y_df.sort_values(by=['unique_id', 'ds'])
        return Y_df

    @staticmethod
    def _nonzero_indexes_by_row(M):
        return [np.nonzero(M[row,:])[0] for row in range(len(M))]

    @staticmethod
    def load_item_data(item_id, dataset='Favorita200', directory='./data'):
        # Load data
        data_info = FavoritaInfo[dataset]
        Y_df, S_df, tags = FavoritaData.load(directory=directory,
                                             group=dataset)

        # Parse and augment data
        # + hack geographic hier_id to treat it as unique_id
        Y_df['ds'] = pd.to_datetime(Y_df['ds'])
        Y_df = Y_df[Y_df.item_id==item_id]
        Y_df = Y_df.rename(columns={'hier_id': 'unique_id'})
        Y_df = FavoritaHierarchicalDataset._sort_hier_df(Y_df=Y_df, S_df=S_df)

        # Obtain indexes for plots and evaluation
        hier_levels = ['Overall'] + list(tags.keys())
        hier_idxs = [np.arange(len(S_df))] +\
            [S_df.index.get_indexer(tags[level]) for level in list(tags.keys())]
        hier_linked_idxs = FavoritaHierarchicalDataset._nonzero_indexes_by_row(S_df.values.T)

        # MinT along other methods require a positive definite covariance matrix
        # for the residuals, when dealing with 0s as residuals the methods break
        # data is augmented with minimal normal noise to avoid this error.
        Y_df['y'] = Y_df['y'] + np.random.normal(loc=0.0, scale=0.01, size=len(Y_df))

        # Final output
        data = dict(Y_df=Y_df, S_df=S_df, tags=tags,
                    # Hierarchical idxs
                    hier_idxs=hier_idxs,
                    hier_levels=hier_levels,
                    hier_linked_idxs=hier_linked_idxs,
                    # Dataset Properties
                    horizon=data_info.horizon,
                    freq=data_info.freq,
                    seasonality=data_info.seasonality)
        return data

In [5]:
data = FavoritaHierarchicalDataset.load_item_data(item_id=1916577, directory = './data/favorita', dataset='Favorita200')

100%|██████████| 930M/930M [00:21<00:00, 43.4MiB/s] 
INFO:datasetsforecast.utils:Successfully downloaded favorita-grocery-sales-forecasting2.zip, 930159981, bytes.
INFO:datasetsforecast.utils:Decompressing zip file...
INFO:datasetsforecast.utils:Successfully decompressed data/favorita/favorita-grocery-sales-forecasting2.zip
INFO:datasetsforecast.utils:Decompressing zip file...
INFO:datasetsforecast.utils:Successfully decompressed data/favorita/holidays_events.csv.zip
INFO:datasetsforecast.utils:Decompressing zip file...
INFO:datasetsforecast.utils:Successfully decompressed data/favorita/items.csv.zip
INFO:datasetsforecast.utils:Decompressing zip file...
INFO:datasetsforecast.utils:Successfully decompressed data/favorita/oil.csv.zip
INFO:datasetsforecast.utils:Decompressing zip file...
INFO:datasetsforecast.utils:Successfully decompressed data/favorita/sample_submission.csv.zip
INFO:datasetsforecast.utils:Decompressing zip file...
INFO:datasetsforecast.utils:Successfully decompressed da

saved train.csv to train.feather for fast access


In [6]:
def run_baselines(intervals_method, item_id=1916577, dataset='Favorita200', verbose=False, seed=0):
    with CodeTimer('Read and Parse data   ', verbose):
        data = FavoritaHierarchicalDataset.load_item_data(item_id=item_id,
                                                     dataset=dataset, directory = './data/favorita')
        Y_df = data['Y_df'][["unique_id", 'ds', 'y']]
        S_df, tags = data['S_df'], data['tags']
        horizon = data['horizon']
        seasonality = data['seasonality']
        freq = data['freq']

        # Train/Test Splits
        Y_test_df  = Y_df.groupby('unique_id').tail(horizon)
        Y_train_df = Y_df.drop(Y_test_df.index)
        Y_test_df  = Y_test_df.set_index('unique_id')
        Y_train_df = Y_train_df.set_index('unique_id')

        dataset_str = f'{dataset} item_id={item_id}, h={horizon} '
        dataset_str += f'n_series={len(S_df)}, n_bottom={len(S_df.columns)} \n'
        dataset_str += f'test ds=[{min(Y_test_df.ds), max(Y_test_df.ds)}] '
        print(dataset_str)

    with CodeTimer('Fit/Predict Model	  ', verbose):
        # Read to avoid unnecesary AutoARIMA computation
        item_path = f'./data/{dataset}/{item_id}'
        os.makedirs(item_path, exist_ok=True)
        yhat_file = f'{item_path}/Y_hat.csv'
        yfitted_file = f'{item_path}/Y_fitted.csv'
        yrec_file = f'{item_path}/{intervals_method}_rec.csv'

        if os.path.exists(yhat_file):
            Y_hat_df = pd.read_csv(yhat_file)
            Y_fitted_df = pd.read_csv(yfitted_file)

        else:
            if not os.path.exists(f'./data/{dataset}'):
                os.makedirs(f'./data/{dataset}')			
            fcst = StatsForecast(
                df=Y_train_df, 
                models=[AutoARIMA(season_length=seasonality)],
                fallback_model=[Naive()],
                freq=freq, 
                n_jobs=-1
            )
            Y_hat_df = fcst.forecast(h=horizon, fitted=True, level=LEVEL)
            Y_fitted_df = fcst.forecast_fitted_values()

            Y_hat_df = Y_hat_df.reset_index()
            Y_fitted_df = Y_fitted_df.reset_index()
            Y_hat_df.to_csv(yhat_file, index=False)
            Y_fitted_df.to_csv(yfitted_file, index=False)

        Y_hat_df = Y_hat_df.set_index('unique_id')
        Y_fitted_df = Y_fitted_df.set_index('unique_id')

    with CodeTimer('Reconcile Predictions ', verbose):
        if is_strictly_hierarchical(S=S_df.values.astype(np.float32), 
            tags={key: S_df.index.get_indexer(val) for key, val in tags.items()}):
            reconcilers = [
                BottomUp(),
                TopDown(method='average_proportions'),
                TopDown(method='proportion_averages'),
                MinTrace(method='ols'),
                MinTrace(method='mint_shrink', mint_shr_ridge=1e-6),
                #ERM(method='reg_bu', lambda_reg=100) # Extremely inneficient
                ERM(method='closed')
            ]
        else:
            reconcilers = [
                BottomUp(),
                MinTrace(method='ols'),
                MinTrace(method='mint_shrink', mint_shr_ridge=1e-6),
                #ERM(method='reg_bu', lambda_reg=100) # Extremely inneficient
                ERM(method='closed')
            ]
        
        hrec = HierarchicalReconciliation(reconcilers=reconcilers)
        Y_rec_df = hrec.bootstrap_reconcile(Y_hat_df=Y_hat_df,
                                            Y_df=Y_fitted_df,
                                            S_df=S_df, tags=tags,
                                            level=LEVEL,
                                            intervals_method=intervals_method,
                                            num_samples=10, num_seeds=10)

        # Matching Y_test/Y_rec/S index ordering
        Y_test_df = Y_test_df.reset_index()
        Y_test_df.unique_id = Y_test_df.unique_id.astype('category')
        Y_test_df.unique_id = Y_test_df.unique_id.cat.set_categories(S_df.index)
        Y_test_df = Y_test_df.sort_values(by=['unique_id', 'ds'])

        Y_rec_df = Y_rec_df.reset_index()
        Y_rec_df.unique_id = Y_rec_df.unique_id.astype('category')
        Y_rec_df.unique_id = Y_rec_df.unique_id.cat.set_categories(S_df.index)
        Y_rec_df = Y_rec_df.sort_values(by=['seed', 'unique_id', 'ds'])

        Y_rec_df.to_csv(yrec_file, index=False)

        return Y_rec_df, Y_test_df, Y_train_df

In [7]:
# Parse execution parameters
verbose = True
intervals_method = 'bootstrap'
dataset = 'Favorita200'

assert intervals_method in ['bootstrap', 'normality', 'permbu'], \
    "Select `--intervals_method` from ['bootstrap', 'normality', 'permbu']"

available_datasets = ['Favorita200', 'Favorita500', 'FavoritaComplete']
assert dataset in available_datasets, \
    "Select `--dataset` from ['Favorita200', 'Favorita500', 'FavoritaComplete']"

LEVEL = np.arange(0, 100, 2)
qs = [[50-lv/2, 50+lv/2] for lv in LEVEL]
QUANTILES = np.sort(np.concatenate(qs)/100)

# Run experiments
Y_all_df, S_df, tags = FavoritaData.load(directory='./data/favorita/', group='Favorita200')
items = Y_all_df.item_id.unique()

print('\n')
print(f'{intervals_method.upper()} {dataset} statistical baselines evaluation \n')
#for item_id in items:
for item_id in [112830, 1916577]:
    Y_rec_df, Y_test_df, Y_train_df = run_baselines(item_id=item_id, dataset=dataset,
                                                    intervals_method=intervals_method,
                                                    verbose=verbose)
    print('\n')



BOOTSTRAP Favorita200 statistical baselines evaluation 

Favorita200 item_id=112830, h=34 n_series=93, n_bottom=54 
test ds=[(Timestamp('2017-07-13 00:00:00'), Timestamp('2017-08-15 00:00:00'))] 
Code block 'Read and Parse data   ' took:	0.60365 seconds
Code block 'Fit/Predict Model	  ' took:	30.04792 seconds
Code block 'Reconcile Predictions ' took:	38.54011 seconds


Favorita200 item_id=1916577, h=34 n_series=93, n_bottom=54 
test ds=[(Timestamp('2017-07-13 00:00:00'), Timestamp('2017-08-15 00:00:00'))] 
Code block 'Read and Parse data   ' took:	0.61150 seconds
Code block 'Fit/Predict Model	  ' took:	29.67945 seconds
Code block 'Reconcile Predictions ' took:	38.78811 seconds


